In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import anndata as ad
import matplotlib.pyplot as plt

from scgpt.preprocess import Preprocessor
from scipy.io import mmread

# Multiple sclerosis

Download `Normalised counts files (MatrixMarket archive)` at https://www.ebi.ac.uk/gxa/sc/experiments/E-HCAD-35/downloads

- retain nuclei with at least 500 genes and 1000 transcripts expressed
- Genes expressed in less than three nuclei were filtered out
- Mitochondrial RNA genes were filtered out as well to exclude transcripts originating from outside the nucleus and avoid biases introduced by nuclei isolation and ultracentrifugation
- UMIs were normalized to the total UMIs per nucleus and log-transformed

In [ ]:
excluded_celltypes = ('B cell', 'T cell', 'oligodendrocyte B')
celltype_rep = 'Factor Value[inferred cell type - authors labels]'
batch_rep = 'Sample Characteristic[disease]'
data_is_raw = False
n_bins = 51

In [103]:
# load the expression matrix
expr_mtx = mmread("datasets/multiple_sclerosis/E-HCAD-35.aggregated_filtered_counts.mtx").T.tocsr()  # to support slicing

# load experiment design
exp_design = pd.read_csv("datasets/multiple_sclerosis/ExpDesign-E-HCAD-35.tsv", sep='\t', index_col=0)

# load cell names
cols = pd.read_csv("datasets/multiple_sclerosis/E-HCAD-35.aggregated_filtered_counts.mtx_cols", header=None, names=['cell_id'], sep='\t')
print((cols['cell_id'] == exp_design.index).all())

# load gene names
rows = pd.read_csv("datasets/multiple_sclerosis/E-HCAD-35.aggregated_filtered_counts.mtx_rows", header=None, names=['feature_id'], sep="\t", usecols=[0])
gene_info = pd.read_csv("datasets/gene_info.csv", index_col=0)
rows = rows.merge(gene_info[['feature_id', 'feature_name']], how='left')
rows.index = rows.index.astype(str)  # to prevent the warnings: Transforming to str index

# create the AnnData objects and save
adata = ad.AnnData(X=expr_mtx, obs=exp_design, var=rows)
adata_prep = adata[~adata.obs[celltype_rep].isin(excluded_celltypes), ~adata.var['feature_name'].isna()].copy()
adata_prep.var_names = adata_prep.var['feature_name']
adata_prep.var.drop(columns=['feature_name'], inplace=True)
# sc.pp.highly_variable_genes(adata_prep, flavor="cell_ranger", subset=True, n_top_genes=3000, batch_key=batch_rep)
adata_control = adata_prep[adata_prep.obs[batch_rep].str.startswith('C')].copy()  # controls
adata_ms = adata_prep[adata_prep.obs['Sample Characteristic[individual]'].str.startswith('MS')].copy()  # patients
# adata_control.write_h5ad("datasets/multiple_sclerosis/controls.h5ad")
# adata_ms.write_h5ad("datasets/multiple_sclerosis/patients.h5ad")

True


- health vs disease
- health as train, disease as test, each saved as a `.h5ad` format file respectively
- common cell types
- cell type labels saved in `.obs['celltype']`
- gene names are `.var.index`
- raw counts
- filter low-quality cells and genes, but do not normalize
- use `seurat_v3_paper` and `batch_key` to select **3000** HVGs and subset
- adata.X is `ndarray`

geneformer

change tie to random selection for determining pred labels.